In [ ]:
import dlib
import cv2

In [ ]:
# Load image
img = cv2.imread("emily.jpg")

# Convert to dlib
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# dlib face detection
detector = dlib.get_frontal_face_detector()
detections = detector(img, 1)

# Find landmarks
sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
faces = dlib.full_object_detections()
for det in detections:
    faces.append(sp(img, det))

# Bounding box and eyes
bb = [i.rect for i in faces]
bb = [((i.left(), i.top()),
       (i.right(), i.bottom())) for i in bb]                            # Convert out of dlib format

right_eyes = [[face.part(i) for i in range(36, 42)] for face in faces]
right_eyes = [[(i.x, i.y) for i in eye] for eye in right_eyes]          # Convert out of dlib format

left_eyes = [[face.part(i) for i in range(42, 48)] for face in faces]
left_eyes = [[(i.x, i.y) for i in eye] for eye in left_eyes]            # Convert out of dlib format

# Display
imgd = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)             # Convert back to OpenCV
for i in bb:
    cv2.rectangle(imgd, i[0], i[1], (255, 0, 0), 5)     # Bounding box

for eye in right_eyes:
    cv2.rectangle(imgd, (max(eye, key=lambda x: x[0])[0], max(eye, key=lambda x: x[1])[1]),
                        (min(eye, key=lambda x: x[0])[0], min(eye, key=lambda x: x[1])[1]),
                        (0, 0, 255), 5)
    for point in eye:
        cv2.circle(imgd, (point[0], point[1]), 2, (0, 255, 0), -1)

for eye in left_eyes:
    cv2.rectangle(imgd, (max(eye, key=lambda x: x[0])[0], max(eye, key=lambda x: x[1])[1]),
                        (min(eye, key=lambda x: x[0])[0], min(eye, key=lambda x: x[1])[1]),
                        (0, 255, 0), 5)
    for point in eye:
        cv2.circle(imgd, (point[0], point[1]), 2, (0, 0, 255), -1)

#cv2.imwrite("output.jpg", imgd)

cv2.imshow("output", imgd)
cv2.waitKey(0)
cv2.destroyAllWindows() # close the window

### Other Example

In [1]:
import numpy as np
import cv2
class FaceAligner:
    def __init__(self, predictor, desiredLeftEye=(0.35, 0.35), desiredFaceWidth=256, desiredFaceHeight=None) :
        # store the facial landmark predictor, desired output left eye position, and desired output face width + height
        self.predictor = predictor
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight
        # if the desired face height is None, set it to be the desired face width (normal behavior)
        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth
    def align(self, image, gray, rect) :
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = self.predictor(gray, rect)
        shape = shape_to_np(shape)
        
        # extract the left and right eye (x, y)-coordinates
        (lStart, lEnd) = FACIAL_LANDMARKS_IDXS["left_eye"]
        (rStart, rEnd) = FACIAL_LANDMARKS_IDXS["right_eye"]
        leftEyePts = shape[lStart:lEnd]
        rightEyePts = shape[rStart:rEnd]
        
        # compute the center of mass for each eye
        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
        
        # compute the angle between the eye centroids
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180
        
        # compute the desired right eye x-coordinate based on the desired x-coordinate of the left eye
        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]
        
        # determine the scale of the new resulting image by taking the ratio of the distance between eyes in the *current*
        # image to the ratio of distance between eyes in the *desired* image
        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist
        
        # compute center (x, y)-coordinates (i.e., the median point) between the two eyes in the input image
        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2, (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
        
        # grab the rotation matrix for rotating and scaling the face
        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
        
        # update the translation component of the matrix
        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])
        
        # apply the affine transformation
        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
        return output   # return the aligned face

In [ ]:
# import the necessary packages
import imutils
import dlib
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import glob
import os, sys

# Creating face_cascade object
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
dataset_path = './knn_dlib/'
offset = 10

# ===== create file_name folder under knn_dlib =====
if not os.path.exists(dataset_path + "/aligned" ):
    os.makedirs(dataset_path + "/aligned")

# initialize dlib's face detector (HOG-based) and then create the facial landmark predictor and the face aligner
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=256)

# load each image in folder 
filenames = [img for img in glob.glob("knn_dlib/hyFS/*.jpg")]
filenames.sort() # ADD THIS LINE
images = []
# load each image in folder, resize it, and convert it to grayscale
for img in filenames:
    image = cv2.imread(img)
    image = imutils.resize(image, width=800)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # show the original input image and detect faces in the grayscale
    # image
    #cv2.imshow("Input", image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows() # close the window
    rects = detector(gray, 2)
    # loop over the face detections
    for rect in rects:
        # extract the ROI of the *original* face, then align the face
        # using facial landmarks
        (x, y, w, h) = rect_to_bb(rect)
        faceOrig = imutils.resize(image[y:y + h, x:x + w], width=256)
        faceAligned = fa.align(image, gray, rect)
        #face_section = faceAligned[y-offset:y+h+offset, x-offset:x+w+offset]
        #face_section = cv2.resize(face_section,(150, 150))
    # save each aligned face 
    cv2.imwrite(img, faceAligned)
    # display the output images
    cv2.imshow("Original", faceOrig)
    cv2.imshow("Aligned", faceAligned)
    cv2.waitKey(0)
    cv2.destroyAllWindows() # close the window        

'''
# load sample image, resize it, and convert it to grayscale
image = cv2.imread("emily.jpg")
image = imutils.resize(image, width=800)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# show the original input image and detect faces in the grayscale
# image
cv2.imshow("Input", image)
cv2.waitKey(0)
cv2.destroyAllWindows() # close the window
rects = detector(gray, 2)
# loop over the face detections
for rect in rects:
    # extract the ROI of the *original* face, then align the face
    # using facial landmarks
    (x, y, w, h) = rect_to_bb(rect)
    faceOrig = imutils.resize(image[y:y + h, x:x + w], width=256)
    faceAligned = fa.align(image, gray, rect)
# display the output images
cv2.imshow("Original", faceOrig)
cv2.imshow("Aligned", faceAligned)
cv2.waitKey(0)
cv2.destroyAllWindows() # close the window
'''

In [ ]:
# ========== 2) load image and save its info as a key to compare ==========
def get_face_encodings(face):
    """
        return np.array of face recognition model which contains location, landmarks for face encoding
    """
    bounds = face_detector(face, 1) # detect face rectangles 
    faces_landmarks = [shape_predictor(face, face_bounds) for face_bounds in bounds]
    return [np.array(face_recognition_model.compute_face_descriptor(face, face_pose, 1)) for face_pose in faces_landmarks]

def get_face_matches(known_faces, face):
    """
        return euclidean distance 
    """
    return np.linalg.norm(known_faces - face, axis=1)

In [ ]:
def find_match(known_faces, person_name, face):
    """
        min distance is the best prediction 
    """
    matches = get_face_matches(known_faces, face) 
    print('matches ', matches)
    min_index = matches.argmin() # min distance index
    print('min index ', min_index)
    min_value = matches[min_index] # min distance
    print('min value ', min_value)
    matchPercent = 100 - (min_value * 100) # convert to percentage
    print('matchPercent ', matchPercent, ' person name ', person_name)
    if matchPercent >= 70 : # at least 80% of correction -> change to 70
        return person_name +" {0:.2f}%".format(matchPercent)
    return 'Not Found'

In [ ]:
def load_face_encodings(faces_folder_path):
    """
        Load face images in person's name folder in a separate window 
    """
    image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir(faces_folder_path))
    image_filenames = sorted(image_filenames)
    person_names = []
    for x in image_filenames :
        #print('image file name ', x)
        index = x.find('_')
        person_names.append(x[:index]) # exclude from '_'
    full_paths_to_images = [faces_folder_path + x for x in image_filenames]
    print('full paths to images ', full_paths_to_images)
    face_encodings = []
    
    for path_to_image in full_paths_to_images:
        face = io.imread(path_to_image)
        faces_bounds = face_detector(face, 1)
        if len(faces_bounds) != 1:
            print("Expected one and only one face per image: " + path_to_image + " - it has " + str(len(faces_bounds)))
        face_bounds = faces_bounds[0]
        face_landmarks = shape_predictor(face, face_bounds)
        face_encoding = np.array(face_recognition_model.compute_face_descriptor(face, face_landmarks, 1))
        face_encodings.append(face_encoding)
    #print('face encoding result ', face_encoding)

    # ===== save this data into numpy array file & text file =====
    np.save(dataset_path + person_names[0] + '.npy', face_encoding)
    print("data successfully saved at " + dataset_path + person_names[0] + '.npy')
    return person_names[0]

In [ ]:
def data_preparation(data_dir) :
    """
        Data Preparation by loading npy file 
    """
    face_data = []
    for dataset in os.listdir(data_dir):
        #print("Loaded "+ dataset)
        if dataset.endswith('.npy'):
            data_item = np.load(dataset_path + dataset)
            face_data.append(data_item)
    return face_data

In [ ]:
# ========== 3) initialize webcam to compare key image features ==========

data_dir = os.path.expanduser('./knn_dlib')
faces_folder_path = data_dir + '/hy/'  # just need to change folder's name to encode 
        
# ===== use timer to get how long it takes to load face encoding =====
start_loading = time.time()
person_name = load_face_encodings(faces_folder_path)
end_loading = time.time()
total_loading = end_loading - start_loading
print('Took {0:.2f} seconds to load face encodings'.format(total_loading))

# ===== load npy file =====
face_data = data_preparation(data_dir)

# ===== initialize webcam =====
camera = cv2.VideoCapture(0)
old_faces = []
while True:
    ret, frame = camera.read()
    if not ret:
        break
    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)    
    faces = face_detector(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_rects = face_cascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5, minSize = (50, 50), flags = cv2.CASCADE_SCALE_IMAGE)
    if len(old_faces) < len(faces) :
        old_faces = []
        for face in faces :
            tracker = dlib.correlation_tracker()
            tracker.start_track(frame, face)
            old_faces.append(tracker)
    else :
        for i, tracker in enumerate(old_faces) :
            quality = tracker.update(frame)
            if quality > 7 :
                pos = tracker.get_position()
                pos = dlib.rectangle(int(pos.left()), int(pos.top()), int(pos.right()), int(pos.bottom()))
                face = frame[pos.top():pos.top() + pos.bottom(), pos.left():pos.left() + pos.right()]
                start = time.time() 
                face_encodings_in_image = get_face_encodings(face)
                if (face_encodings_in_image) :
                    match = find_match(face_data, person_name, face_encodings_in_image[0])
                    end = time.time()
                    total = end - start
                    print('Encoding Image Match Found took {0:.2f} seconds'.format(total))
                    cv2.putText(frame, match, (pos.left()-50, pos.top()-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                else : 
                    end = time.time()
                    total = end - start
                    print('Match Not Found took {0:.2f} seconds'.format(total))
                    cv2.putText(frame, "Unknown", (pos.left()-15, pos.top()-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                cv2.rectangle(frame, (pos.left(), pos.top()), (pos.right(), pos.bottom()), (0, 255, 255), 2)
            else:
                old_faces.pop(i)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
camera.release()
cv2.destroyAllWindows()